# Rastreamento para Diferentes Tipos de Execuções

### Tipos de Execuções

LangSmith suporta muitos tipos diferentes de Execuções - você pode especificar que tipo sua Execução é no decorador @traceable. Os tipos de execuções são:

- LLM: Invoca um LLM
- Retriever: Recupera documentos de bancos de dados ou outras fontes
- Tool: Executa ações com chamadas de função
- Chain: Tipo padrão; combina múltiplas Execuções em um processo maior
- Prompt: Hidrata um prompt para ser usado com um LLM
- Parser: Extrai dados estruturados

### Configuração

In [ ]:
# Você pode defini-las diretamente!
import os
os.environ["GOOGLE_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

In [ ]:
# Ou você pode usar um arquivo .env
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

### Execuções LLM para Modelos de Chat

LangSmith fornece renderização e processamento especiais para rastreamentos LLM. Para aproveitar ao máximo este recurso, você deve registrar seus rastreamentos LLM em um formato específico.

Para modelos estilo chat, as entradas devem ser uma lista de mensagens em formato compatível com OpenAI, representadas como dicionários Python ou objetos TypeScript. Cada mensagem deve conter as chaves role e content.

A saída é aceita em qualquer um dos seguintes formatos:

- Um dicionário/objeto que contém a chave choices com um valor que é uma lista de dicionários/objetos. Cada dicionário/objeto deve conter a chave message, que mapeia para um objeto de mensagem com as chaves role e content.
- Um dicionário/objeto que contém a chave message com um valor que é um objeto de mensagem com as chaves role e content.
- Uma tupla/array de dois elementos, onde o primeiro elemento é o role e o segundo elemento é o content.
- Um dicionário/objeto que contém as chaves role e content.

A entrada para sua função deve ser nomeada messages.

Você também pode fornecer os seguintes campos de metadados para ajudar o LangSmith a identificar o modelo e calcular custos. Se usar LangChain ou funções helper Gemini, esses campos serão preenchidos automaticamente corretamente.
- ls_provider: O provedor do modelo, ex. "google", "anthropic", etc.
- ls_model_name: O nome do modelo, ex. "gemini-1.5-flash", "claude-3-opus-20240307", etc.

In [ ]:
from langsmith import traceable

inputs = [
  {"role": "system", "content": "Você é um assistente útil."},
  {"role": "user", "content": "Gostaria de reservar uma mesa para dois."},
]

output = {
  "choices": [
      {
          "message": {
              "role": "assistant",
              "content": "Claro, que horas você gostaria de reservar a mesa?"
          }
      }
  ]
}

# Também pode usar um dos:
# output = {
#     "message": {
#         "role": "assistant",
#         "content": "Claro, que horas você gostaria de reservar a mesa?"
#     }
# }
#
# output = {
#     "role": "assistant",
#     "content": "Claro, que horas você gostaria de reservar a mesa?"
# }
#
# output = ["assistant", "Claro, que horas você gostaria de reservar a mesa?"]

@traceable(
  run_type="llm",
  metadata={
      "ls_provider": "google",
      "ls_model_name": "gemini-1.5-flash"
  }
)
def chat_model(messages: list):
  return output

chat_model(inputs)

### Lidando com Execuções LLM de Streaming

Para streaming, você pode "reduzir" as saídas para o mesmo formato da versão não-streaming. Isso é atualmente suportado apenas em Python.

In [ ]:
def _reduce_chunks(chunks: list):
    all_text = "".join([chunk["choices"][0]["message"]["content"] for chunk in chunks])
    return {"choices": [{"message": {"content": all_text, "role": "assistant"}}]}

@traceable(
    run_type="llm",
    metadata={"ls_provider": "google", "ls_model_name": "gemini-1.5-flash"},
    reduce_fn=_reduce_chunks
)
def my_streaming_chat_model(messages: list):
    for chunk in ["Olá, " + messages[1]["content"]]:
        yield {
            "choices": [
                {
                    "message": {
                        "content": chunk,
                        "role": "assistant",
                    }
                }
            ]
        }

list(
    my_streaming_chat_model(
        [
            {"role": "system", "content": "Você é um assistente útil. Por favor, cumprimente o usuário."},
            {"role": "user", "content": "polly o papagaio"},
        ],
    )
)

### Execuções de Retriever + Documentos

Muitas aplicações LLM requerem buscar documentos de bancos de dados vetoriais, grafos de conhecimento ou outros tipos de índices. Rastreamentos de retriever são uma maneira de registrar os documentos que são recuperados pelo retriever. LangSmith fornece renderização especial para etapas de recuperação em rastreamentos para facilitar o entendimento e diagnóstico de problemas de recuperação. Para que as etapas de recuperação sejam renderizadas corretamente, alguns pequenos passos precisam ser dados.

1. Anote a etapa do retriever com run_type="retriever".
2. Retorne uma lista de dicionários Python ou objetos TypeScript da etapa do retriever. Cada dicionário deve conter as seguintes chaves:
    - page_content: O texto do documento.
    - type: Isso deve sempre ser "Document".
    - metadata: Um dicionário python ou objeto TypeScript contendo metadados sobre o documento. Esses metadados serão exibidos no rastreamento.

In [ ]:
from langsmith import traceable

def _convert_docs(results):
  return [
      {
          "page_content": r,
          "type": "Document",
          "metadata": {"foo": "bar"}
      }
      for r in results
  ]

@traceable(
    run_type="retriever"
)
def retrieve_docs(query):
  # Retriever retornando documentos fictícios hardcoded.
  # Em produção, isso poderia ser um banco de dados vetorial real ou outro índice de documentos.
  contents = ["Conteúdo do documento 1", "Conteúdo do documento 2", "Conteúdo do documento 3"]
  return _convert_docs(contents)

retrieve_docs("Consulta do usuário")

### Chamada de Ferramentas

LangSmith tem renderização personalizada para Chamadas de Ferramentas feitas pelo modelo para deixar claro quando ferramentas fornecidas estão sendo usadas.

In [ ]:
from langsmith import traceable
from typing import List, Optional
import json
import sys
sys.path.append('../../')
from gemini_utils import call_gemini_with_tools

@traceable(
  run_type="tool"
)
def get_current_temperature(location: str, unit: str):
    return 65 if unit == "Fahrenheit" else 17

@traceable(
    run_type="llm",
    metadata={
        "ls_provider": "google",
        "ls_model_name": "gemini-1.5-flash"
    }
)
def call_gemini(
    messages: List[dict], tools: Optional[List[dict]]
) -> str:
  return call_gemini_with_tools("gemini-1.5-flash", messages, tools, 0)

@traceable(run_type="chain")
def ask_about_the_weather(inputs, tools):
  response = call_gemini(inputs, tools)
  # Para Gemini, a resposta já contém a função chamada e resultado
  return response

tools = [
    {
      "type": "function",
      "function": {
        "name": "get_current_temperature",
        "description": "Obter a temperatura atual para um local específico",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "A cidade e estado, ex., São Paulo, SP"
            },
            "unit": {
              "type": "string",
              "enum": ["Celsius", "Fahrenheit"],
              "description": "A unidade de temperatura a usar. Infira isso da localização do usuário."
            }
          },
          "required": ["location", "unit"]
        }
      }
    }
]
inputs = [
  {"role": "system", "content": "Você é um assistente útil."},
  {"role": "user", "content": "Qual é o clima hoje em São Paulo?"},
]

ask_about_the_weather(inputs, tools)